In [78]:
from surprise import KNNWithMeans, KNNBasic, SlopeOne, NMF, SVD, KNNWithZScore
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

In [16]:
movies = pd.read_csv('movies.dat',
            header=None, sep='::', engine='python')
movies.columns = ['movieId','title','genres']
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [17]:
ratings = pd.read_csv('ratings.dat',
            header=None, sep='::', engine='python')
ratings.columns = ['userId','movieId','rating','timestamp']
ratings.head()

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [18]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [20]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),5.0
1,6.0,Toy Story (1995),4.0
2,8.0,Toy Story (1995),4.0
3,9.0,Toy Story (1995),5.0
4,10.0,Toy Story (1995),5.0


In [68]:
# сделаем  небольшую выборку чтобы было удобнее подбираь модель
dsample = dataset.sample(100000)

In [69]:
sreader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))
sdata = Dataset.load_from_df(dsample, reader)

In [85]:
trainset, testset = train_test_split(sdata, test_size=.15)

In [77]:
algo = KNNWithMeans(k=80, sim_options={'name': 'cosine', 'user_based': False})
algo.fit(trainset)
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9804


0.9804467319478669

In [55]:
algo = SlopeOne()
algo.fit(trainset)
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 1.0250


1.0249839093146844

In [79]:
algo = SVD()
algo.fit(trainset)
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9522


0.952174480117024

In [60]:
algo = NMF()
algo.fit(trainset)
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 1.0311


1.0310738329238407

In [91]:
algo = KNNWithZScore(k=100, sim_options={'name': 'cosine', 'user_based': False})
algo.fit(trainset)
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9805


0.9805468411150359

In [92]:
# попробуем наиболее удачные варианты на всем объеме данных
reader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))
data = Dataset.load_from_df(dataset, reader)

In [93]:
trainset, testset = train_test_split(data, test_size=.15)

In [83]:
algo = SVD()
algo.fit(trainset)
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8694


0.8693592084700351